In [1]:
import os
import json
import pandas as pd
import re
import unicodedata
import numpy as np

In [2]:
FILE = '../data/raw/applicants.json'

def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Applicants
data = load_json(FILE)
df = pd.DataFrame([
    {
        'candidato_id': cid,
        **data.get('infos_basicas', {}),
        **data.get('formacao_e_idiomas', {}),
        **data.get('informacoes_profissionais', {}),
        'cv_pt': data.get('cv_pt'),
    }
    for cid, data in data.items()
])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42482 entries, 0 to 42481
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   candidato_id                 42482 non-null  object
 1   telefone_recado              42482 non-null  object
 2   telefone                     42482 non-null  object
 3   objetivo_profissional        42482 non-null  object
 4   data_criacao                 42482 non-null  object
 5   inserido_por                 42482 non-null  object
 6   email                        42482 non-null  object
 7   local                        42482 non-null  object
 8   sabendo_de_nos_por           42482 non-null  object
 9   data_atualizacao             42482 non-null  object
 10  codigo_profissional          42482 non-null  object
 11  nome                         42482 non-null  object
 12  nivel_academico              42482 non-null  object
 13  nivel_ingles                 42

In [4]:
df.columns

Index(['candidato_id', 'telefone_recado', 'telefone', 'objetivo_profissional',
       'data_criacao', 'inserido_por', 'email', 'local', 'sabendo_de_nos_por',
       'data_atualizacao', 'codigo_profissional', 'nome', 'nivel_academico',
       'nivel_ingles', 'nivel_espanhol', 'outro_idioma', 'titulo_profissional',
       'area_atuacao', 'conhecimentos_tecnicos', 'certificacoes',
       'outras_certificacoes', 'remuneracao', 'nivel_profissional', 'cv_pt',
       'instituicao_ensino_superior', 'cursos', 'ano_conclusao', 'outro_curso',
       'qualificacoes', 'experiencias'],
      dtype='object')

In [5]:
for col in df.columns:
    if df[col].dtype == 'object':
        print(f"Column '{col}' has {df[col].nunique()} unique values.")

Column 'candidato_id' has 42482 unique values.
Column 'telefone_recado' has 1 unique values.
Column 'telefone' has 41026 unique values.
Column 'objetivo_profissional' has 8368 unique values.
Column 'data_criacao' has 41708 unique values.
Column 'inserido_por' has 63 unique values.
Column 'email' has 41482 unique values.
Column 'local' has 721 unique values.
Column 'sabendo_de_nos_por' has 14 unique values.
Column 'data_atualizacao' has 41761 unique values.
Column 'codigo_profissional' has 42482 unique values.
Column 'nome' has 40686 unique values.
Column 'nivel_academico' has 22 unique values.
Column 'nivel_ingles' has 6 unique values.
Column 'nivel_espanhol' has 6 unique values.
Column 'outro_idioma' has 33 unique values.
Column 'titulo_profissional' has 8368 unique values.
Column 'area_atuacao' has 838 unique values.
Column 'conhecimentos_tecnicos' has 3190 unique values.
Column 'certificacoes' has 270 unique values.
Column 'outras_certificacoes' has 884 unique values.
Column 'remune

In [6]:
df = df.drop(
    [
    'telefone_recado',
    'telefone',
    'data_criacao',
    'inserido_por',
    'email',
    'data_atualizacao',
    'codigo_profissional',
    'outro_curso',
    'qualificacoes',
    'experiencias',
    'remuneracao'
    ]
    , axis=1)

In [7]:
for column in df.columns:
    if df[column].nunique() < 35:
        print(f"Column '{column}' unique values: {df[column].unique()}")

Column 'sabendo_de_nos_por' unique values: ['' 'Outros' 'Anúncio' 'Site de Empregos' 'Indeed'
 'Indicação de colaborador' 'Nosso site' 'Indicação de amigo'
 'Escolas/Faculdades' 'Indicação de cliente' 'Evento universitário'
 'ONG/PAT/CAT' 'Feira de RH' 'Plaqueiro']
Column 'nivel_academico' unique values: ['' 'Ensino Superior Incompleto' 'Ensino Superior Completo'
 'Ensino Técnico Completo' 'Ensino Superior Cursando'
 'Pós Graduação Completo' 'Pós Graduação Cursando'
 'Pós Graduação Incompleto' 'Ensino Técnico Cursando'
 'Ensino Fundamental Completo' 'Ensino Médio Completo'
 'Doutorado Completo' 'Mestrado Completo' 'Mestrado Incompleto'
 'Mestrado Cursando' 'Doutorado Incompleto' 'Ensino Técnico Incompleto'
 'Ensino Médio Cursando' 'Ensino Médio Incompleto'
 'Ensino Fundamental Cursando' 'Doutorado Cursando'
 'Ensino Fundamental Incompleto']
Column 'nivel_ingles' unique values: ['' 'Nenhum' 'Intermediário' 'Básico' 'Avançado' 'Fluente']
Column 'nivel_espanhol' unique values: ['' 'Nenhum

In [8]:
df = df.drop(
     [
     'sabendo_de_nos_por'
     ]
     , axis=1)

In [23]:
df.nivel_profissional.value_counts(normalize=True) * 100

nivel_profissional
                          99.811685
Sênior                     0.065910
Analista                   0.040017
Pleno                      0.025893
Especialista               0.018832
Júnior                     0.016478
Líder                      0.009416
Técnico de Nível Médio     0.004708
Gerente                    0.002354
Outro                      0.002354
Estagiário                 0.002354
Name: proportion, dtype: float64

In [24]:
df = df.drop( 'nivel_profissional', axis=1)

In [9]:
df.columns

Index(['candidato_id', 'objetivo_profissional', 'local', 'nome',
       'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma',
       'titulo_profissional', 'area_atuacao', 'conhecimentos_tecnicos',
       'certificacoes', 'outras_certificacoes', 'nivel_profissional', 'cv_pt',
       'instituicao_ensino_superior', 'cursos', 'ano_conclusao'],
      dtype='object')

In [10]:
for col in df.columns:
    print(df[col].value_counts(normalize=True) * 100)
    print("=" * 60)

candidato_id
5999     0.002354
31000    0.002354
5983     0.002354
5982     0.002354
5981     0.002354
           ...   
31006    0.002354
31005    0.002354
31004    0.002354
31003    0.002354
31002    0.002354
Name: proportion, Length: 42482, dtype: float64
objetivo_profissional
                                           33.934372
Java                                        1.555953
Gerente de Projetos                         1.428840
Suporte                                     1.275834
Desenvolvedor                               1.113413
                                             ...    
Desenvolvedor Dot Net                       0.002354
Administrativa/logística e ou comercial     0.002354
BigData                                     0.002354
Gerente TI                                  0.002354
Analista de planejamento                    0.002354
Name: proportion, Length: 8368, dtype: float64
local
                                  59.964220
São Paulo, São Paulo              15.26

In [11]:
df.head()

,candidato_id,objetivo_profissional,local,nome,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,titulo_profissional,area_atuacao,conhecimentos_tecnicos,certificacoes,outras_certificacoes,nivel_profissional,cv_pt,instituicao_ensino_superior,cursos,ano_conclusao
0,31000,,,Carolina Aparecida,,,,-,,,,,,,assistente administrativo\n\n\nsantosbatista\n...,NaN,NaN,NaN
1,31001,Analista Administrativo,"São Paulo, São Paulo",Eduardo Rios,Ensino Superior Incompleto,Nenhum,Nenhum,-,Analista Administrativo,Administrativa,,,,,formação acadêmica\nensino médio (2º grau) em ...,NaN,NaN,NaN
2,31002,Administrativo | Financeiro,"São Paulo, São Paulo",Pedro Henrique Carvalho,Ensino Superior Completo,Intermediário,Básico,Português - Fluente,Administrativo | Financeiro,Administrativa,,"MS [77-418] MOS: Microsoft Office Word 2013, M...",,,objetivo: área administrativa | financeira\n\n...,,Administração de Empresas,2012
3,31003,Área administrativa,"São Paulo, São Paulo",Thiago Barbosa,Ensino Superior Incompleto,Nenhum,Nenhum,-,Área administrativa,Administrativa,,,,,formação\nensino médio completo\ninformática i...,NaN,NaN,NaN
4,31004,,,Diogo das Neves,,,,-,,,,,,,última atualização em 09/11/2021\n­ sp\n\nensi...,NaN,NaN,NaN


# Tratamentos para o modelo

* objetivo_profissional
* local (separar cidade/estado)
* nivel_academico (feature engineering)
* nivel_ingles (buscar no cv_pt e feature engineering)
* nivel_espanhol (buscar no cv_pt e feature engineering)
* remuneracao (normalizar ou feature engineering)
* nivel_profissional (feature engineering)

In [12]:
df.objetivo_profissional.value_counts(normalize=True) * 100

objetivo_profissional
                                           33.934372
Java                                        1.555953
Gerente de Projetos                         1.428840
Suporte                                     1.275834
Desenvolvedor                               1.113413
                                             ...    
Desenvolvedor Dot Net                       0.002354
Administrativa/logística e ou comercial     0.002354
BigData                                     0.002354
Gerente TI                                  0.002354
Analista de planejamento                    0.002354
Name: proportion, Length: 8368, dtype: float64

In [13]:
df.local.value_counts(normalize=True) * 100

local
                                  59.964220
São Paulo, São Paulo              15.265289
Rio de Janeiro, Rio de Janeiro     1.996140
Fortaleza, Ceará                   1.876089
Campinas, São Paulo                1.388824
                                    ...    
Capixaba, Acre                     0.002354
Sanclerlândia, Goiás               0.002354
Mato Grosso                        0.002354
São Marcos, Rio Grande do Sul      0.002354
Palmas, Paraná                     0.002354
Name: proportion, Length: 721, dtype: float64

In [14]:
def normalizar_local(local):
    # Garante que estamos trabalhando com uma string
    if not isinstance(local, str):
        return pd.Series(['Não informado', 'Não informado'])

    # Remove o padrão numérico do início (ex: '59.964220')
    # r'^\d+\.\d+' significa: no início da string (^), encontre um ou mais dígitos (\d+),
    # seguidos por um ponto literal (\.), seguido por um ou mais dígitos (\d+).
    texto_limpo = re.sub(r'^\d+\.\d+', '', local).strip()

    # Se após a limpeza não sobrar nada, retorna como não informado
    if not texto_limpo:
        return pd.Series(['Não informado', 'Não informado'])

    # Verifica se há uma vírgula para separar cidade e estado
    if ',' in texto_limpo:
        partes = texto_limpo.split(',', 1) # Divide apenas na primeira vírgula
        cidade = partes[0].strip()
        estado = partes[1].strip()
        return pd.Series([cidade, estado])
    else:
        # Se não houver vírgula, assume que é a cidade
        cidade = texto_limpo.strip()
        estado = 'Não informado'
        return pd.Series([cidade, estado])
    
df[['cidade', 'estado']] = df['local'].apply(normalizar_local)

In [15]:
df.estado.value_counts(normalize=True) * 100

estado
Não informado          60.948166
São Paulo              25.773269
Rio de Janeiro          2.739984
Minas Gerais            2.302151
Ceará                   2.109129
Bahia                   1.080458
Paraná                  1.016901
Distrito Federal        0.706181
Pernambuco              0.668518
Rio Grande do Sul       0.659103
Santa Catarina          0.572007
Goiás                   0.270703
Maranhão                0.263641
Espirito Santo          0.157714
Paraíba                 0.141236
Amazonas                0.089450
Pará                    0.084742
Rio Grande do Norte     0.063556
Alagoas                 0.061202
Mato Grosso             0.058848
Sergipe                 0.058848
Piauí                   0.051787
Mato Grosso do Sul      0.047079
Tocantins               0.032955
Rondônia                0.016478
Amapá                   0.014124
Acre                    0.007062
Roraima                 0.004708
Name: proportion, dtype: float64

In [16]:
df.nivel_academico.value_counts(normalize=True) * 100

nivel_academico
                                 80.947225
Ensino Superior Completo          8.318817
Pós Graduação Completo            4.267690
Ensino Superior Cursando          2.111483
Ensino Superior Incompleto        0.981592
Ensino Médio Completo             0.943929
Pós Graduação Cursando            0.680288
Ensino Técnico Completo           0.529636
Mestrado Completo                 0.386046
Pós Graduação Incompleto          0.350737
Ensino Técnico Cursando           0.127113
Mestrado Incompleto               0.077680
Mestrado Cursando                 0.063556
Ensino Fundamental Completo       0.056495
Ensino Técnico Incompleto         0.040017
Doutorado Completo                0.028247
Ensino Médio Incompleto           0.023539
Ensino Médio Cursando             0.021185
Doutorado Incompleto              0.018832
Ensino Fundamental Cursando       0.009416
Doutorado Cursando                0.009416
Ensino Fundamental Incompleto     0.007062
Name: proportion, dtype: float64

In [17]:
df.nivel_academico.unique()

array(['', 'Ensino Superior Incompleto', 'Ensino Superior Completo',
       'Ensino Técnico Completo', 'Ensino Superior Cursando',
       'Pós Graduação Completo', 'Pós Graduação Cursando',
       'Pós Graduação Incompleto', 'Ensino Técnico Cursando',
       'Ensino Fundamental Completo', 'Ensino Médio Completo',
       'Doutorado Completo', 'Mestrado Completo', 'Mestrado Incompleto',
       'Mestrado Cursando', 'Doutorado Incompleto',
       'Ensino Técnico Incompleto', 'Ensino Médio Cursando',
       'Ensino Médio Incompleto', 'Ensino Fundamental Cursando',
       'Doutorado Cursando', 'Ensino Fundamental Incompleto'],
      dtype=object)

In [18]:
def mapear_escolaridade_ordinal(coluna_nivel_academico: pd.Series) -> pd.Series:
    mapeamento_ordinal = {
        # Fundamental
        'Ensino Fundamental Incompleto': 1,
        'Ensino Fundamental Cursando': 2,
        'Ensino Fundamental Completo': 3,
        # Médio
        'Ensino Médio Incompleto': 4,
        'Ensino Médio Cursando': 5,
        'Ensino Médio Completo': 6,
        # Técnico
        'Ensino Técnico Incompleto': 7,
        'Ensino Técnico Cursando': 8,
        'Ensino Técnico Completo': 9,
        # Superior
        'Ensino Superior Incompleto': 10,
        'Ensino Superior Cursando': 11,
        'Ensino Superior Completo': 12,
        # Pós-Graduação / Especialização
        'Pós Graduação Incompleto': 13,
        'Pós Graduação Cursando': 14,
        'Pós Graduação Completo': 15,
        # Mestrado
        'Mestrado Incompleto': 16,
        'Mestrado Cursando': 17,
        'Mestrado Completo': 18,
        # Doutorado
        'Doutorado Incompleto': 19,
        'Doutorado Cursando': 20,
        'Doutorado Completo': 21
    }

    # Aplica o mapeamento, preenche valores não encontrados com 0 e converte para inteiro.
    coluna_ordinal = coluna_nivel_academico.map(mapeamento_ordinal).fillna(0).astype(int)

    return coluna_ordinal

df['ordem_escolaridade'] = mapear_escolaridade_ordinal(df['nivel_academico'])

In [ ]:
def obter_nivel_idioma_pd(df: pd.DataFrame, coluna_primaria: str) -> pd.DataFrame:
    df_processado = df.copy()
    
    if 'ingles' in coluna_primaria.lower():
        idioma = 'ingles'
        regex_com_niveis = r"ingles\s*[:,\s]*\s*(basico|tecnico|intermediario|avancado|fluente)"
        regex_presenca = r"(?:ingles|english)"
    elif 'espanhol' in coluna_primaria.lower():
        idioma = 'espanhol'
        regex_com_niveis = r"espanhol\s*[:,\s]*\s*(basico|tecnico|intermediario|avancado|fluente)"
        regex_presenca = r"(?:espanhol|spanish)"
    else:
        raise ValueError(f"Não foi possível inferir o idioma da coluna '{coluna_primaria}'.")

    mapa_primario = {'Nenhum': 0, 'Básico': 1, 'Técnico': 2, 'Intermediário': 3, 'Avançado': 4, 'Fluente': 5}
    mapa_fallback = {'basico': 1, 'tecnico': 2, 'intermediario': 3, 'avancado': 4, 'fluente': 5}

    nivel_primario = pd.Series(index=df.index, dtype='float64') 
    if coluna_primaria in df_processado.columns:
        coluna_com_nans = df_processado[coluna_primaria].replace(r'^\s*$', np.nan, regex=True)
        nivel_primario = coluna_com_nans.str.title().map(mapa_primario)

    coluna_fallback_texto = 'cv_pt'
    nivel_fallback = pd.Series(index=df.index, dtype='float64')
    if coluna_fallback_texto in df_processado.columns:
        
        cv_lower = df_processado[coluna_fallback_texto].str.lower().fillna('')
        
        cv_normalizado = cv_lower.apply(
            lambda texto: ''.join(c for c in unicodedata.normalize('NFKD', texto) if not unicodedata.combining(c))
        )
        cv_normalizado = cv_normalizado.str.replace('\n', ' ', regex=False)

        niveis_extraidos = cv_normalizado.str.extract(regex_com_niveis, expand=True)[0]
        nivel_numerico_extraido = niveis_extraidos.map(mapa_fallback)
        
        presenca_detectada = cv_normalizado.str.contains(regex_presenca, na=False).astype(int)
        
        nivel_fallback = nivel_numerico_extraido.fillna(presenca_detectada)

    nivel_unificado = nivel_primario.fillna(nivel_fallback)
    df_processado[f"nivel_{idioma}_num"] = nivel_unificado.fillna(0).astype('Int8')

    return df_processado

In [26]:
df = obter_nivel_idioma_pd(df, 'nivel_ingles')
df = obter_nivel_idioma_pd(df, 'nivel_espanhol')

resultado_final_candidato = df[df['candidato_id'] == '31008']

print("--- Resultado final para o candidato 31008 (Após Correção Definitiva) ---")
print(resultado_final_candidato[['candidato_id', 'nivel_ingles', 'nivel_espanhol', 'nivel_ingles_num', 'nivel_espanhol_num']])

--- Resultado final para o candidato 31008 (Após Correção Definitiva) ---
  candidato_id nivel_ingles nivel_espanhol  nivel_ingles_num  \
8        31008                                             1   

   nivel_espanhol_num  
8                   0  


In [27]:
df.columns

Index(['candidato_id', 'objetivo_profissional', 'local', 'nome',
       'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma',
       'titulo_profissional', 'area_atuacao', 'conhecimentos_tecnicos',
       'certificacoes', 'outras_certificacoes', 'cv_pt',
       'instituicao_ensino_superior', 'cursos', 'ano_conclusao', 'cidade',
       'estado', 'ordem_escolaridade', 'nivel_ingles_num',
       'nivel_espanhol_num'],
      dtype='object')